In [ ]:
# The packages
import pandas as pd
from pathlib import Path
import zipfile

from pathlib import Path
from datetime import date
import time, zipfile, io
import shutil, subprocess

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [10]:
# 1. Charger le CSV brut
# attention à l'encodage : utf-8 ou latin-1 selon ton fichier
df = pd.read_csv("../data/Incendies.csv", 
                 sep=";", 
                 skiprows=2,     # saute les 2 premières lignes (métadonnées)
                 engine="python")

In [12]:
df.head()

,Année,Numéro,Département,Code INSEE,Nom de la commune,Date de première alerte,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),...,Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Précision des surfaces,Type de peuplement,Nature,Décès ou bâtiments touchés,Nombre de décès,Nombre de bâtiments totalement détruits,Nombre de bâtiments partiellement détruits,Précision de la donnée
0,2024,5,66,66043,Casteil,2024-01-01 12:02:00,2000,2000,0.0,0.0,...,NaN,NaN,Estimées,NaN,Involontaire (particulier),Non,NaN,NaN,NaN,NaN
1,2024,27351,77,77186,Fontainebleau,2024-01-01 13:48:00,2,2,NaN,0.0,...,NaN,NaN,Estimées,NaN,Involontaire (particulier),Non,NaN,NaN,NaN,NaN
2,2024,20474,11,11322,Roquefort-des-Corbières,2024-01-01 20:42:00,100,10,90.0,0.0,...,NaN,NaN,Mesurées,1.0,NaN,Non,NaN,NaN,NaN,NaN
3,2024,353,2B,2B029,Barbaggio,2024-01-03 11:36:00,1946500,0,1946500.0,0.0,...,NaN,NaN,Mesurées,1.0,Involontaire (travaux),Non,NaN,NaN,NaN,NaN
4,2024,122,2B,2B124,Ghisoni,2024-01-03 12:59:00,2000,0,2000.0,0.0,...,NaN,NaN,Estimées,NaN,NaN,Non,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 24 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Année                                       1366 non-null   int64  
 1   Numéro                                      1366 non-null   int64  
 2   Département                                 1366 non-null   object 
 3   Code INSEE                                  1366 non-null   object 
 4   Nom de la commune                           1366 non-null   object 
 5   Date de première alerte                     1366 non-null   object 
 6   Surface parcourue (m2)                      1366 non-null   int64  
 7   Surface forêt (m2)                          1366 non-null   int64  
 8   Surface maquis garrigues (m2)               1033 non-null   float64
 9   Autres surfaces naturelles hors forêt (m2)  1324 non-null   float64
 10  Surfaces agr

In [ ]:
def _find_chrome_binary():
    """
    Essaie de trouver un binaire Chrome/Chromium courant.
    """
    for cmd in ["google-chrome", "google-chrome-stable", "chromium", "chromium-browser", "/snap/bin/chromium"]:
        if shutil.which(cmd):
            return shutil.which(cmd)
    return None

def scrape_bdiff_incendies_jupyter(start: date, end: date, out_dir: Path, year_label: str = None, headless: bool = True) -> Path:
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    chrome_opts = Options()
    prefs = {
        "download.default_directory": str(out_dir.resolve()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,
    }
    chrome_opts.add_experimental_option("prefs", prefs)

    # -- Headless: certains Chromes n'aiment pas --headless=new
    if headless:
        chrome_opts.add_argument("--headless")  # <-- plus compatible
    # -- Flags Linux courants pour environnements restreints
    chrome_opts.add_argument("--no-sandbox")
    chrome_opts.add_argument("--disable-dev-shm-usage")
    chrome_opts.add_argument("--window-size=1400,900")

    # -- Tenter de pointer vers un binaire Chrome/Chromium si trouvé
    chrome_bin = _find_chrome_binary()
    if chrome_bin:
        chrome_opts.binary_location = chrome_bin

    # Instanciation du driver (driver auto-matché via webdriver-manager)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
    wait = WebDriverWait(driver, 25)
    def _wait_dom(): wait.until(lambda d: d.execute_script("return document.readyState") == "complete")

    try:
        driver.get("https://bdiff.agriculture.gouv.fr/incendies")
        _wait_dom()

        # éviter les nouveaux onglets
        driver.execute_script("""
            for (const a of document.querySelectorAll('a[target="_blank"], a[target="_BLANK"]')) a.target = '_self';
            for (const f of document.querySelectorAll('form[target="_blank"], form[target="_BLANK"]')) f.target = '_self';
        """)

        # Remplir dates
        s_iso, e_iso = start.isoformat(), end.isoformat()
        date_inputs = driver.find_elements(By.CSS_SELECTOR, "input[type='date']")
        if len(date_inputs) >= 2:
            date_inputs[0].clear(); date_inputs[0].send_keys(s_iso)
            date_inputs[1].clear(); date_inputs[1].send_keys(e_iso)
        else:
            df = driver.find_element(By.XPATH, "//input[not(@type='hidden') and (contains(translate(@id,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'de') or contains(translate(@name,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'de') or contains(translate(@placeholder,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'début') or contains(translate(@placeholder,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'debut') or contains(translate(@placeholder,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'from'))]")
            dt = driver.find_element(By.XPATH, "//input[not(@type='hidden') and (contains(translate(@id,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'a') or contains(translate(@name,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'a') or contains(translate(@placeholder,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'fin') or contains(translate(@placeholder,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'to'))]")
            driver.execute_script("arguments[0].value=arguments[1]; arguments[0].dispatchEvent(new Event('change'));", df, s_iso)
            driver.execute_script("arguments[0].value=arguments[1]; arguments[0].dispatchEvent(new Event('change'));", dt, e_iso)

        # Filtrer
        for xp in ["//button[contains(., 'Filtrer')]", "//input[@type='submit' and contains(@value,'Filtrer')]", "//button[contains(., 'Rechercher')]", "//*[contains(text(),'Filtrer')]"]:
            try:
                btn = wait.until(EC.element_to_be_clickable((By.XPATH, xp)))
                btn.click(); break
            except Exception:
                pass

        # Attente souple résultats
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//*[self::table or contains(@class,'table') or contains(.,'Résultats') or contains(.,'resultats')]")))
        except Exception:
            pass

        before = {p.name for p in out_dir.glob("*")}

        # CSV
        for xp in ["//a[contains(.,'CSV')]", "//button[contains(.,'CSV')]", "//a[contains(@href,'csv') or contains(@href,'export')]", "//*[contains(text(),'CSV')]"]:
            try:
                el = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xp)))
                el.click(); break
            except Exception:
                pass

        # Attente téléchargement
        t0 = time.time(); target = None
        while time.time() - t0 < 180:
            if any(out_dir.glob("*.crdownload")): time.sleep(0.5); continue
            new_files = [p for p in out_dir.glob("*") if p.name not in before]
            if new_files:
                new_files.sort(key=lambda p: p.stat().st_mtime, reverse=True)
                for p in new_files:
                    if p.suffix.lower() in {".zip", ".csv"}:
                        target = p; break
                if target: break
            time.sleep(0.5)
        if not target:
            raise TimeoutError("Téléchargement non détecté.")

        label = (year_label or f"{s_iso}_{e_iso}")
        final = out_dir / f"bdiff_incendies_{label}{target.suffix.lower()}"
        if final.exists(): final.unlink()
        target.rename(final)
        return final

    finally:
        driver.quit()


In [29]:
OUT_DIR = Path("bdiff_exports")
for y in range(2006, 2026):
    print(f"== {y} ==")
    p = scrape_bdiff_incendies_jupyter(date(y,1,1), date(y,12,31), OUT_DIR, year_label=str(y), headless=True)
    print("OK:", p.name)


== 2006 ==


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5599e71804e3 <unknown>
#1 0x5599e6eafc76 <unknown>
#2 0x5599e6ed6757 <unknown>
#3 0x5599e6ed5029 <unknown>
#4 0x5599e6f13ccc <unknown>
#5 0x5599e6f1347f <unknown>
#6 0x5599e6f0ade3 <unknown>
#7 0x5599e6ee02dd <unknown>
#8 0x5599e6ee134e <unknown>
#9 0x5599e71403e4 <unknown>
#10 0x5599e71443d7 <unknown>
#11 0x5599e714eb20 <unknown>
#12 0x5599e7145023 <unknown>
#13 0x5599e71131aa <unknown>
#14 0x5599e71696b8 <unknown>
#15 0x5599e7169847 <unknown>
#16 0x5599e7179243 <unknown>
#17 0x7fbd9f15eaa4 <unknown>
#18 0x7fbd9f1ebc3c <unknown>


In [31]:
def extract_csvs(zip_folder: Path):
    for zf in sorted(zip_folder.glob("*.zip")):
        with zipfile.ZipFile(zf) as z:
            for name in z.namelist():
                if name.lower().endswith(".csv"):
                    out_csv = zf.with_suffix(".csv")
                    with z.open(name) as src, open(out_csv, "wb") as dst:
                        dst.write(src.read())
                    print("→ extrait:", out_csv.name)

extract_csvs(Path("bdiff_exports"))


# Fusion (streaming, mémoire maîtrisée)
final_out = Path("incendies_2006_2025.csv")
wrote = False
for csvf in sorted(Path("bdiff_exports").glob("*.csv")):
    df_iter = pd.read_csv(csvf, sep=";", chunksize=30_000)  # adapte le séparateur si besoin
    for chunk in df_iter:
        chunk.to_csv(final_out, sep=";", index=False, header=(not wrote), mode=("w" if not wrote else "a"))
        wrote = True

print("Fichier fusionné →", final_out.resolve())



Fichier fusionné → /home/kahina/laplateforme/ProjetIncendie/notebooks/incendies_2006_2025.csv
